# Facebook data 📈

Directory selection

In [ ]:
from PyQt5 import QtWidgets

import json
import os
from pathlib import Path 

def get_directory():
    app = QtWidgets.QApplication([dir])
    return QtWidgets.QFileDialog.getExistingDirectory(None, "Select messages directory")

dirname = get_directory()

conversation_dirs = os.listdir(dirname)
print('Printing conversations...')
print(conversation_dirs)

First of all we need to fix encoding of message.json files. To do this, we will encode message to **latin-1** and decode to **utf8**. Fixed Json files are created in tmp folder in root folder of jupyter notebook file.

In [ ]:
def fixJsons(srcFolder, conversationDirs, destFolder):
    if not os.path.exists(destFolder):
        os.makedirs(destFolder)

    for convdir in conversation_dirs:
            src = '{}/{}/message.json'.format(srcFolder, convdir)
            dst = '{}/{}.json'.format(destFolder,convdir)

            try:
                src_file = open(src, 'r')
            except Exception:
                continue

            jsonToFix = json.load(src_file)

            for i in jsonToFix['messages']:
                try:
                    i['sender_name'] = i['sender_name'].encode('latin-1').decode('utf8')
                    i['content'] = i['content'].encode('latin-1').decode('utf8')
                except Exception:
                    pass

            jsonToFix['title'] = jsonToFix['title'].encode('latin-1').decode('utf8')

            with open(dst, 'w', encoding='utf8') as fout:
                json.dump(jsonToFix, fout, indent=2, ensure_ascii=False)

            print('Source {} generated {}'.format(convdir,dst))
            
fixJsons(dirname,conversation_dirs,'tmp')

Count messages in all conversations

In [ ]:
def countMessages(srcDir):
    conversation_counts = {}

    fixed_jsons = os.listdir(srcDir)

    for jsfile in fixed_jsons:
        fixed_json = json.load(open('{}/{}'.format(srcDir,jsfile), 'r', encoding='utf-8'))
        conversation_counts[str(fixed_json["title"])] = len(fixed_json["messages"])
  
    return conversation_counts

def countMessagesFiltered(srcDir, filterAmount):
    filtered_total = 0
    filtered_counts = {}
    
    fixed_jsons = os.listdir(srcDir)
    
    for jsfile in fixed_jsons:
        fixed_json = json.load(open('{}/{}'.format(srcDir,jsfile), 'r', encoding='utf-8'))

        count = len(fixed_json["messages"])

        if count < filterAmount:
            filtered_total += count
        else:
            filtered_counts[str(fixed_json["title"])] = count

        filtered_counts["Other"] = filtered_total
    
    return filtered_counts

conversation_counts = countMessages('tmp')
filtered_counts = countMessagesFiltered('tmp',1000)

print(conversation_counts)
print(filtered_counts)

Export dictionary to csv file

In [ ]:
def exportToCsv(srcdict, path):
    with open(path,'w',encoding='utf-8') as out_file:
        for k,v in srcdict.items():
            out_file.write('{};{}\n'.format(k,v))

exportToCsv(conversation_counts,'raw.csv')
exportToCsv(filtered_counts,'raw.csv')
        
print("Done writing files")

List all hashtags

In [ ]:
def findHashTags(srcDir):
    hashtag_lists = {}
    fixed_jsons = os.listdir(srcDir)

    for jsfile in fixed_jsons:
        fixed_json = json.load(open('{}/{}'.format(srcDir,jsfile), 'r', encoding='utf-8'))
        print('Analyzing file {} '.format(jsfile), end='', flush=True)
        name = fixed_json["title"]
        listCreated = False

        for msg in fixed_json["messages"]:
            try:
                words = msg['content'].split()

                for word in words:
                    if word.startswith('#'):
                        if not listCreated:
                            hashtag_lists[name] = []
                            listCreated = True
                        if not word in hashtag_lists[name]:
                            hashtag_lists[name].append(word)
            except Exception:
                continue
        if name in hashtag_lists:
            print("Found {} unique hashtags".format(len(hashtag_lists[name])))
        else:
            print("Found 0 hashtags") 
            
    return hashtag_lists

hashtag_lists = findHashTags('tmp')